In [4]:
import os
import pandas as pd

from tqdm import tqdm

#セットアップ
tqdm.pandas()

import sys
sys.path.append("/kaggle/src")
from kaggle_metrics.kaggle_kl_div import compute_each_score

In [5]:
oof_path = "/kaggle/working/exp000_notrepeat_inputs/tf_efficientnet_b0_ns_oof_df_exp000_notrepeat_inputs_stage1.csv"

In [6]:
oof_df = pd.read_csv(oof_path)

In [7]:
oof_df

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,...,other_vote,total_evaluators,target,fold,pred_seizure_vote,pred_lpd_vote,pred_gpd_vote,pred_lrda_vote,pred_grda_vote,pred_other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,1.0,...,0.000000,3,Seizure,0,0.711850,0.059821,0.031907,0.030819,0.019863,0.145740
1,8071080,0,0.0,2593634,0,0.0,92643023,2944,GRDA,0.0,...,0.000000,3,GRDA,0,0.008218,0.037198,0.074058,0.054805,0.541758,0.283964
2,175641702,0,0.0,3250170,0,0.0,2564153803,33127,LRDA,0.0,...,0.000000,3,LRDA,0,0.258192,0.184464,0.087159,0.113383,0.082747,0.274054
3,893864755,0,0.0,4367732,0,0.0,1554550678,2338,Other,0.0,...,0.769231,13,Other,0,0.028503,0.032555,0.044432,0.127075,0.377061,0.390374
4,2184415503,0,0.0,4651057,0,0.0,164296540,47284,Other,0.0,...,1.000000,1,Other,0,0.030377,0.082431,0.343688,0.005772,0.008369,0.529363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20178,918950919,0,0.0,2145646585,0,0.0,3266555556,22147,Other,0.0,...,1.000000,3,Other,4,0.057157,0.013913,0.006017,0.049497,0.120818,0.752598
20179,897183978,0,0.0,2145731520,0,0.0,1724767128,25579,Other,0.0,...,0.666667,3,Other,4,0.106545,0.088707,0.011961,0.169083,0.066964,0.556740
20180,2982417621,0,0.0,2145983945,0,0.0,3989519932,7118,LRDA,0.0,...,0.266667,15,LRDA,4,0.081017,0.028448,0.009771,0.117948,0.395312,0.367504
20181,1306668185,0,0.0,2147312808,0,0.0,1216355904,57480,LPD,0.0,...,0.000000,3,LPD,4,0.101816,0.119881,0.020351,0.162056,0.141520,0.454377


In [8]:
oof_df["valid_score"] = oof_df.progress_apply(compute_each_score, axis=1)

100%|██████████| 20183/20183 [03:12<00:00, 105.07it/s]


In [9]:
oof_path = "/kaggle/working/exp000_notrepeat_inputs/oof_with_valid_score.csv"
oof_df.to_csv(oof_path, index=False)

In [11]:
oof_df.columns

Index(['eeg_id', 'eeg_sub_id', 'eeg_label_offset_seconds', 'spectrogram_id',
       'spectrogram_sub_id', 'spectrogram_label_offset_seconds', 'label_id',
       'patient_id', 'expert_consensus', 'seizure_vote', 'lpd_vote',
       'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote', 'total_evaluators',
       'target', 'fold', 'pred_seizure_vote', 'pred_lpd_vote', 'pred_gpd_vote',
       'pred_lrda_vote', 'pred_grda_vote', 'pred_other_vote', 'valid_score'],
      dtype='object')

In [13]:
for fold in range(5):
    print(f" === fold {fold} ===")
    fold_oof_df = oof_df[oof_df["fold"] == fold]
    fold_score = fold_oof_df["valid_score"].mean()
    print(f"fold{fold} score: {fold_score}")
    display(fold_oof_df["expert_consensus"].value_counts())

 === fold 0 ===
fold0 score: 0.6598619670179203


expert_consensus
Other      1462
Seizure     789
LPD         744
GRDA        427
LRDA        343
GPD         272
Name: count, dtype: int64

 === fold 1 ===
fold1 score: 0.6535199070182584


expert_consensus
Other      1555
Seizure     951
LPD         658
GRDA        357
GPD         277
LRDA        239
Name: count, dtype: int64

 === fold 2 ===
fold2 score: 0.6300183074524038


expert_consensus
Other      1561
Seizure     915
LPD         654
GPD         385
GRDA        340
LRDA        182
Name: count, dtype: int64

 === fold 3 ===
fold3 score: 0.7112945256947493


expert_consensus
Other      1474
Seizure     693
LPD         671
GPD         542
GRDA        419
LRDA        237
Name: count, dtype: int64

 === fold 4 ===
fold4 score: 0.6480249998293086


expert_consensus
Other      1665
Seizure     625
LPD         623
GPD         605
GRDA        387
LRDA        131
Name: count, dtype: int64

In [14]:
for fold in range(5):
    print(f" === fold {fold} ===")
    fold_oof_df = oof_df[oof_df["fold"] == fold]
    display(fold_oof_df.groupby("expert_consensus")["valid_score"].mean())

 === fold 0 ===


expert_consensus
GPD        0.654201
GRDA       0.806606
LPD        0.597986
LRDA       1.535939
Other      0.441256
Seizure    0.664961
Name: valid_score, dtype: float64

 === fold 1 ===


expert_consensus
GPD        0.656758
GRDA       0.881951
LPD        0.712362
LRDA       1.209079
Other      0.454389
Seizure    0.712095
Name: valid_score, dtype: float64

 === fold 2 ===


expert_consensus
GPD        0.578073
GRDA       0.841912
LPD        0.785732
LRDA       1.216351
Other      0.430927
Seizure    0.684867
Name: valid_score, dtype: float64

 === fold 3 ===


expert_consensus
GPD        0.787737
GRDA       1.175029
LPD        0.794439
LRDA       1.515199
Other      0.390549
Seizure    0.697912
Name: valid_score, dtype: float64

 === fold 4 ===


expert_consensus
GPD        0.506019
GRDA       1.349447
LPD        0.797707
LRDA       1.509279
Other      0.432603
Seizure    0.595327
Name: valid_score, dtype: float64